In [19]:
from langchain.document_loaders.csv_loader import CSVLoader
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.llms import OpenAI
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
from langchain import LLMChain
from langchain.chains.question_answering import load_qa_chain

In [20]:
import os

First, initialize your LLM

In [21]:
# TODO: initialize your LLM

os.environ["OPENAI_API_KEY"] = ""
os.environ["OPENAI_API_BASE"] = "https://openai.vocareum.com/v1"

Then, load reviews from tv-reviews.csv

In [22]:
# TODO: load your documents
from langchain.indexes import VectorstoreIndexCreator
from langchain.document_loaders.csv_loader import CSVLoader

loader = CSVLoader(file_path='./tv-reviews.csv')
docs = loader.load()
print(docs)

[Document(page_content="TV Name: Imagix Pro\nReview Title: Amazing Picture Quality\nReview Rating: 9\nReview Text: I recently purchased the Imagix Pro and I am blown away by its picture quality. The colors are vibrant and the images are crystal clear. It feels like I'm watching movies in a theater! The sound is also impressive, creating a truly immersive experience. Highly recommended!", metadata={'source': './tv-reviews.csv', 'row': 0}), Document(page_content="TV Name: Imagix Pro\nReview Title: Impressive Features\nReview Rating: 8\nReview Text: The Imagix Pro is packed with impressive features that enhance my viewing experience. The smart functionality allows me to easily stream my favorite shows and movies. The remote control is user-friendly and has convenient shortcuts. The slim design is sleek and fits perfectly in my living room. The only downside is that the sound could be better, but overall, I'm satisfied.", metadata={'source': './tv-reviews.csv', 'row': 1}), Document(page_co

Split the documents you loaded into smaller chunks

In [23]:
# TODO: use a Text Splitter to split the documents into chunks
from langchain.prompts import PromptTemplate
from langchain import LLMChain
from langchain.llms import OpenAI

model_name="gpt-3.5-turbo"
temperature = 0
llm = OpenAI(model_name=model_name, temperature=temperature, max_tokens = 2000)

splitter = CharacterTextSplitter(chunk_size=1000,chunk_overlap=0)
split_doc=splitter.split_documents(docs)



Now, initialize your embeddings

In [24]:
# TODO: initialize your embeddings model

embeddings = OpenAIEmbeddings()


Initialize your vector db with your embeddings model and populate with your text chunks

In [27]:
# TODO: populate your vector database with the chunks
db = Chroma.from_documents(split_doc,embeddings)

Query your vector database for 5 most semantically similar chunks

In [29]:
query = """
    Based on the reviews in the context, tell me what people liked about the picture quality.
    Make sure you do not paraphrase the reviews, and only use the information provided in the reviews.
    """
# find top 5 semantically similar documents to the query

use_chain_helper = False
if use_chain_helper:
    rag = RetrievalQA.from_chain_type(llm=llm,chain_type="stuff",retriever=db.as_retriever())
    print(rag.run(query))
else:
    similar_docs = db.similarity_search(query,k=5)
    prompt = PromptTemplate(
             template = "{query}\nContext : {context}",
             input_variables = ["query","context"])

chain = load_qa_chain(llm,prompt=prompt,chain_type="stuff")
print(chain.run(input_documents=similar_docs, query=query))


People liked the vibrant colors and crystal clear images of the picture quality. They were impressed by the sharpness and lifelike details, making it feel like they were watching movies in a theater. The colors were described as vibrant and realistic, enhancing the overall viewing experience. The visuals were unmatched, with incredible detail and vibrant colors that brought every scene to life.


Combined, they should provide enough information to answer our question about picture quality

In [ ]:
# query your LLM with the query and the top 5 documents